In [4]:
# importing libraries

import pandas as pd
import dateutil.parser
import json
import gzip
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from collections import defaultdict
from urllib.request import urlopen
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, confusion_matrix, precision_score, recall_score
import math
import time
import datetime
import tarfile
import csv
import random
import scipy.optimize

In [2]:
# 1-time use

#tf = tarfile.open("assignment1.tar.gz")
#tf.extractall()

In [3]:
df = pd.read_csv('trainInteractions.csv.gz', compression = 'gzip')

In [4]:
df.shape

(500000, 4)

In [5]:
# splitting data into train and val

dft = df[:400000]
dfv = df[400000:]

In [6]:
dft.shape

(400000, 4)

In [7]:
dfv.shape

(100000, 4)

# Cook Prediction

In [2]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [8]:
# creating a column that will be used to classify

dfv['cooked'] = 1

<ipython-input-8-e7902c83c0a3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfv['cooked'] = 1


In [9]:
itemsPerUser = defaultdict(set)
for i, d in dfv.iterrows():
    itemsPerUser[d['user_id']].add(d['recipe_id'])

In [10]:
# creating one negative validation entry for every posiive validation entry using set subtraction

for i, d in dfv.iterrows():
    random.seed(0)
    dfv = dfv.append({'user_id': d['user_id'], 'recipe_id': random.sample(set(dfv['recipe_id']) - itemsPerUser[d['user_id']], 1)[0], 
                      'rating': 0, 'cooked':0}, ignore_index=True)

In [11]:
# was previously (100000,4)

dfv.shape

(200000, 5)

In [9]:
# save validation data as it takes a lot of time to generate it (cell 294)

#dfv.to_csv('sampled_validation.csv', index = False)
#dfv = pd.read_csv('sampled_validation.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for i, d in dft.iterrows():
    usersPerItem[d['recipe_id']].add(d['user_id'])
    itemsPerUser[d['user_id']].add(d['recipe_id'])

In [11]:
def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [13]:
dataset = list(parse("trainRecipes.json.gz"))

In [15]:
dfi = pd.DataFrame()
dfi['name'] = [d['name'] for d in dataset]
dfi['minutes'] = [d['minutes'] for d in dataset]
dfi['contributor_id'] = [d['contributor_id'] for d in dataset]
dfi['submitted'] = [d['submitted'] for d in dataset]
dfi['steps'] = [d['steps'] for d in dataset]
dfi['description'] = [d['description'] for d in dataset]
dfi['ingredients'] = [d['ingredients'] for d in dataset]
dfi['recipe_id'] = [d['recipe_id'] for d in dataset]

In [20]:
dfi['recipe_id'] = dfi['recipe_id'].astype('int')

In [17]:
long_string = ''
for d in dfi['ingredients']:
    long_string = long_string + ',' + ','.join(d)

In [18]:
long_list = long_string.split(',')
df_popular = pd.DataFrame({'items':long_list})
popular_items = list(df_popular['items'].value_counts().index[:50])
popular_items

['salt',
 'butter',
 'sugar',
 'onion',
 'water',
 'eggs',
 'olive oil',
 'flour',
 'milk',
 'garlic cloves',
 'pepper',
 'brown sugar',
 'garlic',
 'all-purpose flour',
 'baking powder',
 'egg',
 'salt and pepper',
 'parmesan cheese',
 'lemon juice',
 'baking soda',
 'vegetable oil',
 'vanilla',
 'black pepper',
 'cinnamon',
 'tomatoes',
 'sour cream',
 'garlic powder',
 'vanilla extract',
 'oil',
 'honey',
 'garlic clove',
 'cream cheese',
 'onions',
 'celery',
 'cheddar cheese',
 'unsalted butter',
 'mayonnaise',
 'soy sauce',
 'chicken broth',
 'paprika',
 'extra virgin olive oil',
 'worcestershire sauce',
 'fresh parsley',
 'cornstarch',
 'fresh ground black pepper',
 'parsley',
 'carrots',
 'chili powder',
 'ground cinnamon',
 'bacon']

In [28]:
dfj = pd.merge(dfv, dfi, how =  'left', left_on = 'recipe_id', right_on = 'recipe_id')

In [29]:
dfj.head()

,user_id,recipe_id,date,rating,cooked,name,minutes,contributor_id,submitted,steps,description,ingredients
0,90764166.0,1768679.0,2011-09-10,5.0,1.0,baked german potato salad,55,83301801,2003-09-03,set oven to 350 degrees f\tbutter a 3-quart ba...,this recipe was given to me by a german lady t...,"[bacon, celery, onion, salt, all-purpose flour..."
1,68112239.0,24923981.0,2007-03-30,5.0,1.0,red wine vinaigrette,3,30355466,2006-07-24,pour each ingredient into a small bowl\twith a...,this is something i've been working on for my ...,"[red wine vinegar, olive oil, lemon juice, dri..."
2,32173358.0,57597698.0,2009-01-16,4.0,1.0,cornbread in a round low fat,30,91094577,2008-03-13,"in one bowl mix cornmeal , flour , baking powd...",this comes from the new dr. cookie cookbook. i...,"[yellow cornmeal, unbleached all-purpose flour..."
3,30893740.0,16266088.0,2010-06-12,5.0,1.0,orange onion and olive salad munkaczina,20,68479185,2010-05-03,slice oranges crosswise and remove seeds and w...,this is a very simple and refreshing salad fro...,"[oranges, onion, ripe olives, cayenne pepper, ..."
4,69780905.0,62953151.0,2004-02-10,5.0,1.0,cheesy mostaccioli,70,04676371,2003-09-04,prepare the pasta according to the package dir...,this recipe is from an old pampered chef cookb...,"[mostaccioli pasta, ground beef, spaghetti sau..."


In [26]:
dfj['pop_ing'] = 1

In [42]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [17]:
sim  = []
for i, d in dfv.iterrows():
    recipes = set({})
    for recipe in itemsPerUser[d['user_id']]:
        recipes = recipes.union(usersPerItem[recipe])

    if d['recipe_id'] not in usersPerItem:
        sim.append(Jaccard(usersPerItem[d['recipe_id']], recipes))
    else:
        sim.append(0)

In [35]:
# one-time as the above cell took a lot of time

#pd.DataFrame({'sim':sim}).to_csv('sim.csv',index=False)
sim = pd.read_csv('sim.csv')

In [37]:
sim = sim['sim']

In [40]:
sim = sim.tolist()

In [28]:
threshold = .005
acc = []
for thresh in threshold:
    tptn = 0
    for i, d in dfv.iterrows():         
        if sim[i] > thresh:
            if d['cooked'] != 0:
                tptn += 1
        else:
            if d['cooked'] == 0:
                tptn += 1
    acc.append(tptn/dfv.shape[0])

In [29]:
print(str(threshold[acc.index(np.max(acc))]) + ' threshold has the best performance ' + str(np.max(acc)))

0.0010404040404040403 threshold has the best performance 0.640375


In [68]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for i, d in df.iterrows():
    usersPerItem[str(d['recipe_id'])].add(str(d['user_id']))
    itemsPerUser[str(d['user_id'])].add(str(d['recipe_id']))

In [66]:
df_sort = pd.DataFrame(df['recipe_id'].value_counts()).reset_index()
df_sort = df_sort.rename(columns = {'recipe_id':'count','index':'recipe_id'})

popular = set()
count = 0
for i, d in zip(df_sort['count'],df_sort['recipe_id']):
    count += i
    popular.add(str(d))
    if count > df.shape[0] * 0.73:
        break

popularity only - 67.7
similarity - 64.9

In [46]:
accuracies = []
percentiles = np.linspace(0.01,1,100) 

for perc in percentiles:
    
    popular = set()
    count = 0
    for i, d in zip(df_sort['count'],df_sort['recipe_id']):
        count += i
        popular.add(d)
        if count > df.shape[0] * perc:
            break

    tptn = 0
    for i, d in dfv.iterrows():
        if d['recipe_id'] in popular:
            if d['cooked'] == 1:
                tptn += 1
        else:
            if d['cooked'] == 0:
                tptn += 1
                
    accuracies.append(tptn/dfv.shape[0])

In [47]:
print(str(percentiles[accuracies.index(np.max(accuracies))]) + ' percentile has the best performance ' + str(np.max(accuracies)))

0.51 percentile has the best performance 0.65413


In [297]:
# converted Professor's code in a Pandas implementation

dft_sort = pd.DataFrame(dft['recipe_id'].value_counts()).reset_index()
dft_sort = dft_sort.rename(columns = {'recipe_id':'count','index':'recipe_id'})

In [298]:
# we now have our data in descending value_count format

dft_sort.head()

,recipe_id,count
0,74912490,503
1,29147042,467
2,5898774,410
3,37919560,384
4,98124873,367


In [299]:
# inspired by baselines.py

popular = set()
count = 0
for i, d in zip(dft_sort['count'],dft_sort['recipe_id']):
    count += i
    popular.add(d)
    if count > dft.shape[0] * 0.5:
        break

In [300]:
# TP/TN

tptn = 0
for i, d in dfv.iterrows():
    if d['recipe_id'] in popular:
        if d['cooked'] == 1:
            tptn += 1
    else:
        if d['cooked'] == 0:
            tptn += 1

print('Accuracy of baseline model on valiation data: ' + str(tptn/dfv.shape[0]))

Accuracy of baseline model on valiation data: 0.64191


In [301]:
# finding the best threshold / percentile

accuracies = []
percentiles = np.linspace(0.01,1,100) 

for perc in percentiles:
    
    popular = set()
    count = 0
    for i, d in zip(dft_sort['count'],dft_sort['recipe_id']):
        count += i
        popular.add(d)
        if count > dft.shape[0] * perc:
            break

    tptn = 0
    for i, d in dfv.iterrows():
        if d['recipe_id'] in popular:
            if d['cooked'] == 1:
                tptn += 1
        else:
            if d['cooked'] == 0:
                tptn += 1
                
    accuracies.append(tptn/dfv.shape[0])

In [302]:
# Answer 2
print(str(int(100*percentiles[accuracies.index(np.max(accuracies))])) + ' percentile has the best performance ' + str(np.max(accuracies)))

52 percentile has the best performance 0.64294


In [307]:
# calculating the max similarity and storing it in a list

M = []
for i, d in dfv.iterrows():
    sim = []
    for recipe in itemsPerUser[d['user_id']]:
        if d['recipe_id'] in usersPerItem:
            sim.append(Jaccard(usersPerItem[d['recipe_id']], usersPerItem[recipe]))
        else:
            sim.append(0)
    M.append(max(sim, default = 0))

In [333]:
# calculating best accuracy

accuracies = []
#threshold = [1/2**i for i in range(1,10)] + [1/3**i for i in range(1,7)] + [1/5**i for i in range(1,5)] + [1/7**i for i in range(1,4)]
threshold = np.linspace(.001,.02,100)

for thresh in threshold:
    tptn = 0
    for i, d in dfv.iterrows():
        if M[i] > thresh:
            if d['cooked'] == 1:
                tptn += 1
        else:
            if d['cooked'] == 0:
                tptn += 1
                    
    accuracies.append(tptn/dfv.shape[0])

In [336]:
# Answer 3

print(str(threshold[accuracies.index(np.max(accuracies))]) + ' threshold has the best performance ' + str(np.max(accuracies)))

0.010595959595959596 threshold has the best performance 0.619885


In [337]:
# improve the above accuracy, construct 'popular' based on the threshold from Answer 2

popular = set()
count = 0
for i, d in zip(dft_sort['count'],dft_sort['recipe_id']):
    count += i
    popular.add(d)
    if count > dft.shape[0] * 0.52: # from answer 2
        break

In [338]:
# improving the above predictor by incorporating both Jacard and popularity thresholds

tptn = 0
for i, d in dfv.iterrows():
    if M[i] > 0.010595959595959596 and d['recipe_id'] in popular: # from answer 3
        if d['cooked'] == 1:
            tptn += 1
    else:
        if d['cooked'] == 0:
            tptn += 1

# Answer 4
tptn/dfv.shape[0]

0.63059

### Tune again for both the percentile and threshold

In [342]:
# put all the above steps in a function

def q4(percentile, threshold):
    popular = set()
    count = 0
    for i, d in zip(dft_sort['count'],dft_sort['recipe_id']):
        count += i
        popular.add(d)
        if count > dft.shape[0] / percentile:
            break
    
    tptn = 0
    for i, d in dfv.iterrows():
        if M[i] > threshold and d['recipe_id'] in popular:
            if d['cooked'] == 1:
                tptn += 1
        else:
            if d['cooked'] == 0:
                tptn += 1

    return (tptn/dfv.shape[0])

In [343]:
percentiles = [i/10 for i in range(10, 20)]
acc = []
for percentile in percentiles:
    acc.append(q4(percentile, 0.010595959595959596)) # from Answer 3

In [344]:
print(str(percentiles[acc.index(np.max(acc))]) + ' threshold has the best performance ' + str(np.max(acc)))

1.7 threshold has the best performance 0.63381


In [345]:
# using the above threshold

thresholds = [i/5000 for i in range(50, 100, 10)]
acc = []
for threshold in thresholds:
    acc.append(q4(1.7, threshold)) # from the above cell

In [346]:
print(str(thresholds[acc.index(np.max(acc))]) + ' threshold has the best performance ' + str(np.max(acc)))

0.01 threshold has the best performance 0.633955


In [75]:
# use the entire dataset to train the model with the above values 1.7 and 0.01

popular = set()
count = 0
for i, d in zip(df_sort['count'],df_sort['recipe_id']):
    count += i
    popular.add(str(d))
    if count > df.shape[0] *.73:
        break

In [ ]:
usersPerItem1 = defaultdict(set)
itemsPerUser1 = defaultdict(set)
for i, d in dft.iterrows():
    usersPerItem1[str(d['recipe_id'])].add(str(d['user_id']))
    itemsPerUser1[str(d['user_id'])].add(str(d['recipe_id']))

In [76]:
# write out the predictions
# Kaggle - 'Thyronorm'

predictions = open("predictions_Made.txt", 'w')
for l in open("stub_Made.txt"):
    if l.startswith("user_id"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    #similar = []
    #for recipe in itemsPerUser1[u]:
     #   if i in usersPerItem1:
      #      similar.append(Jaccard(usersPerItem1[i], usersPerItem1[recipe]))
       # else:
        #    similar.append(0)
    r = np.random.choice(np.arange(0, 2), p=[.8,.2])
    if i in popular:
    #if max(similar, default = 0) > 0.005:
        predictions.write(u + '-' + i + ",1\n")
    else:
        #predictions.write(u + '-' + i + ",\n")
        predictions.write(u + '-' + i + "," + str(r) + "\n")

predictions.close()

In [57]:
c1

5736

In [58]:
c2

14264

# Rating Prediction

In [6]:
path = "renttherunway_final_data.json.gz" 
f = gzip.open(path, 'rt', encoding="utf8") 
data = []
for l in f: 
    d = json.loads(l) 
    data.append(d)
f.close()

In [1]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

In [4]:
recipeData = []
for user, recipe, date, rating in readCSV("trainInteractions.csv.gz"):
    recipeData.append([user, recipe, int(rating)])

In [5]:
# 80-20 split for training and validation

numTraining = int(len(recipeData) * 0.80)
recipeDataTrain = recipeData[:numTraining]
recipeDataValid = recipeData[numTraining:]

In [6]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

for user_id,recipe_id,r in recipeDataTrain:
    reviewsPerUser[user_id].append(r)
    reviewsPerItem[recipe_id].append(r)

In [7]:
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

In [8]:
alpha = sum([d[2] for d in recipeDataTrain]) / len(recipeDataTrain)
alpha

4.5808

In [9]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [10]:
# defining some basic functions

def prediction(user, item):
    userBias = 0
    itemBias = 0
    if user in userBiases:
        userBias = userBiases[user]
    if item in itemBiases:
        itemBias = itemBiases[item]
    pred = alpha + userBias + itemBias
    if pred > 5:
        pred = 5
    elif pred < 0:
        pred = 0
    return pred
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1])) # as 0th is already alpha
    itemBiases = dict(zip(items, theta[1+nUsers:]))
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(uId, rId) for uId, rId, r in recipeDataTrain]
    cost = mean_squared_error(predictions, labels)
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(recipeDataTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for uId, rId, r in recipeDataTrain:
        pred = prediction(uId, rId)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[uId] += 2/N*diff
        dItemBiases[rId] += 2/N*diff
    for u in userBiases:
        dUserBiases[uId] += 2*lamb*userBiases[uId]
    for i in itemBiases:
        dItemBiases[rId] += 2*lamb*itemBiases[rId]
    dtheta = [dalpha] + [dUserBiases[uId] for uId in users] + [dItemBiases[rId] for rId in items]
    return np.array(dtheta)

In [11]:
# MSE if we always predict mean

alwaysPredictMean = [alpha for d in recipeDataTrain]
labels = [r for uId,rId,r in recipeDataTrain]
mean_squared_error(alwaysPredictMean, labels)

0.89873136

In [12]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems), derivative, args = (labels, 1))

(array([ 4.58080000e+00, -9.99073018e-05, -9.36398828e-06, ...,
        -1.68523934e-06,  1.21634354e-06, -1.68523934e-06]),
 0.8986650519796527,
 {'grad': array([ 2.86762882e-04,  1.72015107e-04,  1.61351631e-05, ...,
          2.90388169e-06, -2.09023906e-06,  2.90347479e-06]),
  'task': 'ABNORMAL_TERMINATION_IN_LNSRCH',
  'funcalls': 44,
  'nit': 1,
  'warnflag': 2})

In [13]:
# MSE on validation when only training data was used in the above step

predictions = [prediction(uId, rId) for uId, rId, r in recipeDataValid]
labelsValid = [r for uId,rId,r in recipeDataValid]
mean_squared_error(predictions, labelsValid)

0.9093902742036257

In [14]:
def writePred(lambdaVal):
    fileName = "predictions_Rated_" + str(lambdaVal) + ".txt"
    predictions = open(fileName, 'w')
    for l in open("stub_Rated.txt"):
        if l.startswith("user_id"):
            predictions.write(l)
            continue
        uId,rId = l.strip().split('-')
        predictions.write(uId + '-' + rId + ',' + str(prediction(uId, rId)) + '\n')
    predictions.close()

In [15]:
# MSE on validation when only training data was used in the above step

powers = [-7,-6,-5,-4,-3,-2,-1]
for power in powers:
    lambda_ = pow(10,power)
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems), derivative, args = (labels, lambda_))
    predictions = [prediction(uId, rId) for uId, rId, r in recipeDataValid]
    labelsValid = [r for uId,rId,r in recipeDataValid]
    mse = mean_squared_error(predictions, labelsValid)
    print("Lambda: %.10f, MSE of validation set = %f" % (lambda_, mse))
    writePred(lambda_)

Lambda: 0.0000001000, MSE of validation set = 0.977422
Lambda: 0.0000010000, MSE of validation set = 0.900194
Lambda: 0.0000100000, MSE of validation set = 0.827652
Lambda: 0.0001000000, MSE of validation set = 0.830766
Lambda: 0.0010000000, MSE of validation set = 0.884592
Lambda: 0.0100000000, MSE of validation set = 0.899688
Lambda: 0.1000000000, MSE of validation set = 0.909171


In [16]:
# further drilling down on lambda from the above results

factors = [.9,2,3]
for factor in factors:
    lambda_ = factor * pow(10,-5)
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems), derivative, args = (labels, lambda_))
    predictions = [prediction(uId, rId) for uId, rId, r in recipeDataValid]
    labelsValid = [r for uId,rId,r in recipeDataValid]
    mse = mean_squared_error(predictions, labelsValid)
    print("Lambda: %.10f, MSE of validation set = %f" % (lambda_, mse))
    writePred(lambda_)

Lambda: 0.0000090000, MSE of validation set = 0.829340
Lambda: 0.0000200000, MSE of validation set = 0.821910
Lambda: 0.0000300000, MSE of validation set = 0.840676


In [17]:
recipeData = []
for user, recipe, date, rating in readCSV("trainInteractions.csv.gz"):
    recipeData.append([user, recipe, int(rating)])

In [18]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

for user_id,recipe_id,r in recipeData:
    reviewsPerUser[user_id].append(r)
    reviewsPerItem[recipe_id].append(r)

In [19]:
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

In [20]:
alpha = sum([d[2] for d in recipeData]) / len(recipeData)
alpha

4.580794

In [21]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [171]:
# defining some basic functions

def prediction(user, item):
    userBias = 0
    itemBias = 0
    if user in userBiases:
        userBias = userBiases[user]
    if item in itemBiases:
        itemBias = itemBiases[item]
    pred = alpha + userBias + itemBias
    if pred > 5:
        pred = 5
    elif pred < 0:
        pred = 0
    return pred
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1])) # as 0th is already alpha
    itemBiases = dict(zip(items, theta[1+nUsers:]))
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(uId, rId) for uId, rId, r in recipeData]
    cost = mean_squared_error(predictions, labels)
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(recipeData)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for uId, rId, r in recipeData:
        pred = prediction(uId, rId)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[uId] += 2/N*diff
        dItemBiases[rId] += 2/N*diff
    for u in userBiases:
        dUserBiases[uId] += 2*lamb*userBiases[uId]
    for i in itemBiases:
        dItemBiases[rId] += 2*lamb*itemBiases[rId]
    dtheta = [dalpha] + [dUserBiases[uId] for uId in users] + [dItemBiases[rId] for rId in items]
    return np.array(dtheta)

In [172]:
# MSE if we always predict mean

alwaysPredictMean = [alpha for d in recipeData]
labels = [r for uId,rId,r in recipeData]
mean_squared_error(alwaysPredictMean, labels)

0.9181778837731382

In [24]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems), derivative, args = (labels, 1))

(array([ 4.58079399e+00, -5.30542108e-05, -7.15649092e-06, ...,
         9.29627600e-07,  9.29627579e-07, -7.20274035e-06]),
 0.9008262946622485,
 {'grad': array([ 2.74042795e-04,  9.56212061e-05,  1.29079697e-05, ...,
         -1.67729030e-06, -1.67668436e-06, -2.18188459e+00]),
  'task': 'ABNORMAL_TERMINATION_IN_LNSRCH',
  'funcalls': 58,
  'nit': 3,
  'warnflag': 2})

In [25]:
# MSE on validation when only training data was used in the above step

predictions = [prediction(uId, rId) for uId, rId, r in recipeDataValid]
labelsValid = [r for uId,rId,r in recipeDataValid]
mean_squared_error(predictions, labelsValid)

0.9093889609810253

In [26]:
lambda_ =  2 * pow(10,-5)
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems), derivative, args = (labels, lambda_))
predictions = [prediction(uId, rId) for uId, rId, r in recipeDataValid]
labelsValid = [r for uId,rId,r in recipeDataValid]
mse = mean_squared_error(predictions, labelsValid)
writePred(lambda_)

In [27]:
mse

0.7169955676706133

In [173]:
userGamma = {}
itemGamma = {}

In [174]:
K = 2

In [175]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [176]:
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [177]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

s = 0
for u in userGamma:
    s += np.mean(userGamma[u])

In [178]:
def inner(x, y):
    try:
        return sum([a*b for a,b in zip(x,y)])
    except TypeError:
        return 0

In [179]:
def prediction(user, item):
    userBias = 0
    itemBias = 0
    userGam = 0
    itemGam = 0
    if user in userBiases:
        userBias = userBiases[user]
        userGam = userGamma[user]
    if item in itemBiases:
        itemBias = itemBiases[item]
        itemGam = itemGamma[item]
    pred = alpha + userBias + itemBias + inner(userGam, itemGam)
    if pred > 5:
        pred = 5
    elif pred < 0:
        pred = 0
    elif abs(round(pred) - pred) <= 0.05:
        pred = round(pred)
    return pred

In [180]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(uId, rId) for uId, rId, r in recipeData]
    # predictions = [prediction(d['user_id'], d['recipe_id']) for d in dataset]
    cost = mean_squared_error(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [181]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(recipeData)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for uId, rId, r in recipeData:
        u,i = uId, rId
        pred = prediction(u, i)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return np.array(dtheta)

In [182]:
mean_squared_error(alwaysPredictMean, labels)

0.9181778837731382

In [183]:
random.seed(0)

In [184]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, 2 * pow(10,-5)), maxiter = 50)

MSE = 0.9181784459821148
MSE = 1.076626
MSE = 0.9525237505803338
MSE = 0.9007268111867569
MSE = 0.900593490331774
MSE = 0.900064559485036
MSE = 0.8980184772690489
MSE = 0.8909484071340253
MSE = 0.863084118735726
MSE = 0.8456047478594836
MSE = 0.829445375110184
MSE = 0.8175261901358734
MSE = 0.806410901370951
MSE = 0.7969980878700884
MSE = 0.779809655044309
MSE = 0.7660567139060801
MSE = 0.7616732697878908
MSE = 0.7570327952460689
MSE = 3.7895198802971097
MSE = 0.7569134485272156
MSE = 0.7544550623254171
MSE = 0.7475203962416929
MSE = 0.7377594208911964
MSE = 0.7282723276677585
MSE = 0.7196074326428364
MSE = 0.7160161129107225
MSE = 0.7141815112438771
MSE = 0.7122471562187573
MSE = 0.7114837260274574
MSE = 0.7138782133680784
MSE = 0.7110880892991449
MSE = 0.7076862788544467
MSE = 0.7048099179871917
MSE = 0.7013582252239368
MSE = 0.7010932583670129
MSE = 0.6986421533077172
MSE = 0.6983463665873955
MSE = 0.6985798947881479
MSE = 0.6982309842949427
MSE = 0.6971055186580295
MSE = 0.69434202

(array([ 4.45216033e+00,  3.74473285e-02, -1.15457790e-01, ...,
        -2.91888259e-03, -4.36140890e-03,  6.81503695e-03]),
 0.7425028623521859,
 {'grad': array([ 1.84246871e-03,  1.08134332e-06, -6.66410379e-07, ...,
         -7.03771777e-08, -1.76579036e-07,  2.73229806e-07]),
  'task': 'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT',
  'funcalls': 63,
  'nit': 50,
  'warnflag': 1})

In [169]:
predictions = [prediction(uId, rId) for uId, rId, r in recipeDataValid]
labelsValid = [r for uId,rId,r in recipeDataValid]
mse = mean_squared_error(predictions, labelsValid)
mse

0.6936918715503715

In [185]:
writePred(lambda_)

In [ ]:
40 - .82318
50 - .82164
55 - .82231
60 - .8221
100 - .824
500 - .84